In [9]:
import os
from together import Together
import json
import time
from tqdm import tqdm

In [10]:
# Filepath to category's dataset
FILEPATH = ""

# Directory to store result files to
RESULTS_DIR = ""

year = "2023"
model = "google/gemma-2-9b-it" # Available models (https://docs.together.ai/docs/chat-models)

In [11]:
PROMPT = '''You are an advanced language model tasked with determining the novelty of research papers in 2024. Your goal is to evaluate and compare the novelty of two research papers based on their titles and abstracts. **The order in which the papers are presented is random and should not influence your evaluation.** Follow these steps to make an accurate and unbiased determination:

### **Step 1: Independent Evaluation of Each Paper**

- **Individually** analyze each research paper’s title and abstract. Treat each paper as if it is the only one under review at that moment, without considering the content of the other paper.
- Evaluate the following aspects independently for each paper:
    - **Novelty of Methodology**: Assess whether the methods used are new and innovative.
    - **Surprisingness of Findings**: Determine if the findings are unexpected or counterintuitive.
    - **Impact on Existing Knowledge**: Evaluate how the research challenges or expands current scientific understanding.
    - **Potential for Future Research**: Consider whether the paper opens up new directions for research.
    - **Relevance to 2024 Scientific Understanding**: Assess how well the paper aligns with or pushes the boundaries of current trends.

### **Step 2: Quantitative Assessment**

- **Independently** assign a score from 1-10 to each research paper for its novelty, with 10 being the most novel. The score should be based solely on the content of the title and abstract.
- Provide a brief justification for each score, using specific quotes and context from the abstract to support your reasoning.

### **Step 3: Reaffirm Independence**

- Before comparing the scores, **reaffirm** that each paper was evaluated independently, without any influence from the order of presentation. Remind yourself that the order of the papers is random and should not affect the final decision.

### **Step 4: Final Comparison**

- Compare the independently assigned scores after reaffirming their independence.
- Determine which paper exhibits greater novelty based on the higher score, and provide the identifier (X or Y) of the more novel paper.
- **Important**: Reflect on whether your final decision would change if the papers were presented in a different order. Ensure that your evaluation is strictly based on content and merit.'''

# test

In [13]:
papers = {
    'paper1' : {
        'title': "Using the power of memes: The Pepper Robot as a communicative\n  facilitator for autistic children (cAESAR2023 workshop)",
        'abstract': "This article describes the preliminary qualitative results of a therapeutic\nlaboratory involving the Pepper robot, as a facilitator, to promote autonomy\nand functional acquisition in autistic children with low support needs (level 1\nsupport). The lab, designed and led by a multidisciplinary team, involved 4\nchildren, aged 11 to 13 years, and was organized in weekly meetings for the\nduration of four months. The following is the result of an in-depth qualitative\nevaluation of the interactions that took place between the children and the\nPepper robot, with the aim of analyzing their effectiveness for the purpose of\npromoting the development of social and communication skills in the\nparticipants. The observations and analyses conducted during the interactions\nprovided valuable insights into the dialogue and communication style employed\nand paved the way for possible strategies to make the robot more empathetic and\nengaging for autistic children.\n"
    },
    'paper2' : {
        'title': "Approximate Counting CSP Solutions Using Partition Function",
        'abstract': "We propose a new approximate method for counting the number of the solutions\nfor constraint satisfaction problem (CSP). The method derives from the\npartition function based on introducing the free energy and capturing the\nrelationship of probabilities of variables and constraints, which requires the\nmarginal probabilities. It firstly obtains the marginal probabilities using the\nbelief propagation, and then computes the number of solutions according to the\npartition function. This allows us to directly plug the marginal probabilities\ninto the partition function and efficiently count the number of solutions for\nCSP. The experimental results show that our method can solve both random\nproblems and structural problems efficiently.\n"
    }
}

In [ ]:
client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))

response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": PROMPT,
        },
        {
            "role": "user",
            "content": f"For the sake of time, please thoroughly evaluate, but only state the identifier (X or Y) of the research paper you believe is more novel. For example if Research Paper X is better response with X and vice versa. No other explanation is needed.\nResearch Paper X Title: {papers['paper1']['title']}\nResearch Paper X Abstract: {papers['paper1']['abstract']}\n\nResearch Paper Y Title: {papers['paper2']['title']}\nResearch Paper Y Abstract: {papers['paper2']['abstract']}"
        }
    ],
)
print(response.choices[0].message.content)

In [ ]:
print(response.json())

# model ___ test

In [ ]:
with open(FILEPATH, 'r') as f:
    data = json.load(f)

data = data['data'][year]
data

In [ ]:
client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))
gaps = ['2', '4', '6', '8', '10']

for gap in gaps:
    results = []
    for papers in tqdm(data[gap]):
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": PROMPT,
                },
                {
                    "role": "user",
                    "content": f"For the sake of time, please thoroughly evaluate, but only state the identifier (X or Y) of the research paper you believe is more novel. For example if Research Paper X is better, respond with X and vice versa. No other explanation is needed.\nResearch Paper X Title: {papers['paper1']['title']}\nResearch Paper X Abstract: {papers['paper1']['abstract']}\n\nResearch Paper Y Title: {papers['paper2']['title']}\nResearch Paper Y Abstract: {papers['paper2']['abstract']}"
                }
            ],
        )
        results.append(response.json())
        time.sleep(1)

        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": PROMPT,
                },
                {
                    "role": "user",
                    "content": f"For the sake of time, please thoroughly evaluate, but only state the identifier (X or Y) of the research paper you believe is more novel. For example if Research Paper X is better, respond with X and vice versa. No other explanation is needed.\nResearch Paper X Title: {papers['paper2']['title']}\nResearch Paper X Abstract: {papers['paper2']['abstract']}\n\nResearch Paper Y Title: {papers['paper1']['title']}\nResearch Paper Y Abstract: {papers['paper1']['abstract']}"
                }
            ],
        )
        results.append(response.json())
        time.sleep(1)

    with open(f'{RESULTS_DIR}/{year}_{gap}_results.json', 'w') as f:
        for result in results:
            f.write(result)
            f.write('\n')